# **Asthma Diagnosis**

### *Team members: Lauren Chin and Lauren Nguyen*

## Dataset

* Source: [Asthma Dataset](https://www.kaggle.com/datasets/rabieelkharoua/asthma-disease-dataset) 
* Format: CSV File
* Contents: This data set contains the patient information for those who are diagnosed with Asthma. Asthma is a chronic lung disease that causes inflamation and tightening of the airways, often making it hard to breath. This dataset contains 2,392 different patients that involve their medical history, symptoms, allergies, and much more.
* Attributes:
    * `PatientID` - a unique identifier for each patient
    * `Age` - age of patient
    * `Gender` - 0 for male and 1 for female
    * `Ethnicity` - encoded as:
        * 0: Caucasian
        * 1: African American
        * 2: Asian
        * 3: Other
    * `EducationLevel` - encoded as:
        * 0: None
        * 1: High School
        * 2: Bachelors
        * 3: Higher
    * `BMI` - body mass index, ranges from 15-40
    * `Smoking` - 0 for no and 1 for yes
    * `PhysicalActivity` - weekly physical activity(hours), ranges 0-10
    * `DietQuality` - ranges from 0-10
    * `SleepQuality` - ranges from 4-10
    * `PollutionExposure` - scored from 0-10
    * `PollenExposure` - scored from 0-10
    * `DustExposure` - scored from 0-10
    * `LungFunctionFEV1` - Forced Expiratory Volume in 1 second (FEV1), ranging from 1.0 to 4.0 liters.
    * `LungFunctionFVC` - Forced Vital Capacity (FVC), ranging from 1.5 to 6.0 liters.
    * `PetAllergy`
    * `FamilyHistoryAsthma` 
    * `HistoryOfAllergies`
    * `Eczema` 
    * `HayFever` 
    * `GastroesophagealReflux`
    * `Wheezing`
    * `Coughing`
    * `ChestTightness`
    * `ShortnessOfBreath`
    * `NighttimeSymptoms`
    * `ExerciseInduced`
    * `DoctorInCharge` - doctor in charge, confidential for all instances
    * **`Diagnosis`** - diagnosis status for asthma patient. 0 for no and 1 for yes

    
    

## Implementation/Technical Merit

### Anticipated Challenges
* Pre-processing:
    * Because there are so many attributes, deciding to use a subset of them or all of them
        * To pare down the attributes, we could randomly select different subsets and compare the model performances to select the "best" attributes
        * However, we instead will likely use unsupervised learning to graph the attributes in comparison to the classifier attribute and determine which attributes have the strongest correlations and use those
    * Discretizing our dataset could be a challenge because there are many attributes with float values that are quite specific (lots of decimal places)
        * For example some of these attributes are: PhysicalActivity (0-10, but float long decimals), DietQuality (0-10, but float long decimals), SleepQuality (4-10, but float long decimals), and others

* Classification: 
    * Because there are so many attributes, making sure that we don't overfit our model
    

## Potential Impact of Results

* Why these results are useful:
    * We are looking at the asthma dataset with the goal of creating classifiers to predict if someone has asthma or not
    * These results are useful because it can help with medical diagnosis
    * While asthma isn't as life threatening as a disease like cancer, detection is still important so measures can be taken to help ease symptoms for patients

* Stakeholders:
    * Doctors (people who have to make the medical diagnosis)
    * Patients (people who could potentially have asthma)
